In [1]:
! hdfs dfs -mkdir -p /user/cloudera/input

In [2]:
! wget  https://s3.amazonaws.com/imcbucket/data/flights/2008.csv

--2020-07-18 01:50:04--  https://s3.amazonaws.com/imcbucket/data/flights/2008.csv
Resolving s3.amazonaws.com... 52.217.40.38
Connecting to s3.amazonaws.com|52.217.40.38|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 689413344 (657M) [binary/octet-stream]
Saving to: “2008.csv.2”

100%[======================================>] 689,413,344 63.2M/s   in 10s     

2020-07-18 01:50:14 (65.1 MB/s) - “2008.csv.2” saved [689413344/689413344]



In [3]:
! hdfs dfs -rm -f /user/cloudera/input/2008.csv

Deleted /user/cloudera/input/2008.csv


In [4]:
! hadoop fs -put 2008.csv /user/cloudera/input

In [5]:
! head -3 ./2008.csv

Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,TailNum,ActualElapsedTime,CRSElapsedTime,AirTime,ArrDelay,DepDelay,Origin,Dest,Distance,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
2008,1,3,4,2003,1955,2211,2225,WN,335,N712SW,128,150,116,-14,8,IAD,TPA,810,4,8,0,,0,NA,NA,NA,NA,NA
2008,1,3,4,754,735,1002,1000,WN,3231,N772SW,128,145,113,2,19,IAD,TPA,810,5,10,0,,0,NA,NA,NA,NA,NA


In [6]:
airline_rdd =  sc.textFile("hdfs:///user/cloudera/input/2008.csv")

In [7]:
airline_rdd.take(2)

[u'Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,TailNum,ActualElapsedTime,CRSElapsedTime,AirTime,ArrDelay,DepDelay,Origin,Dest,Distance,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay',
 u'2008,1,3,4,2003,1955,2211,2225,WN,335,N712SW,128,150,116,-14,8,IAD,TPA,810,4,8,0,,0,NA,NA,NA,NA,NA']

In [8]:
header_text = airline_rdd.first()

In [9]:
airline_noheader_rdd = airline_rdd.filter(lambda row: row != header_text)

In [10]:
airline_noheader_rdd.take(3)

[u'2008,1,3,4,2003,1955,2211,2225,WN,335,N712SW,128,150,116,-14,8,IAD,TPA,810,4,8,0,,0,NA,NA,NA,NA,NA',
 u'2008,1,3,4,754,735,1002,1000,WN,3231,N772SW,128,145,113,2,19,IAD,TPA,810,5,10,0,,0,NA,NA,NA,NA,NA',
 u'2008,1,3,4,628,620,804,750,WN,448,N428WN,96,90,76,14,8,IND,BWI,515,3,17,0,,0,NA,NA,NA,NA,NA']

In [11]:
fieldname_list = header_text.split(',')
def make_row(row):
    fieldvalues_list = row.split(',')
    d = dict(zip(fieldname_list,fieldvalues_list))
    return d

In [12]:
airline_rows_rdd = airline_noheader_rdd.map(make_row)

In [13]:
airline_rows_rdd.take(3)

[{u'ActualElapsedTime': u'128',
  u'AirTime': u'116',
  u'ArrDelay': u'-14',
  u'ArrTime': u'2211',
  u'CRSArrTime': u'2225',
  u'CRSDepTime': u'1955',
  u'CRSElapsedTime': u'150',
  u'CancellationCode': u'',
  u'Cancelled': u'0',
  u'CarrierDelay': u'NA',
  u'DayOfWeek': u'4',
  u'DayofMonth': u'3',
  u'DepDelay': u'8',
  u'DepTime': u'2003',
  u'Dest': u'TPA',
  u'Distance': u'810',
  u'Diverted': u'0',
  u'FlightNum': u'335',
  u'LateAircraftDelay': u'NA',
  u'Month': u'1',
  u'NASDelay': u'NA',
  u'Origin': u'IAD',
  u'SecurityDelay': u'NA',
  u'TailNum': u'N712SW',
  u'TaxiIn': u'4',
  u'TaxiOut': u'8',
  u'UniqueCarrier': u'WN',
  u'WeatherDelay': u'NA',
  u'Year': u'2008'},
 {u'ActualElapsedTime': u'128',
  u'AirTime': u'113',
  u'ArrDelay': u'2',
  u'ArrTime': u'1002',
  u'CRSArrTime': u'1000',
  u'CRSDepTime': u'735',
  u'CRSElapsedTime': u'145',
  u'CancellationCode': u'',
  u'Cancelled': u'0',
  u'CarrierDelay': u'NA',
  u'DayOfWeek': u'4',
  u'DayofMonth': u'3',
  u'DepDela

In [14]:
airline_rows_rdd_repartition = airline_rows_rdd.repartition(10)
airline_rows_rdd_cached = airline_rows_rdd_repartition.cache()

In [15]:
! cp -rf /usr/lib/hive/conf/hive-site.xml /usr/lib/spark/conf/


In [16]:
airline_rows_df = airline_rows_rdd.toDF()

/usr/lib/spark/python/pyspark/sql/context.py:259: UserWarning: Using RDD of dict to inferSchema is deprecated. Use pyspark.sql.Row instead
  warnings.warn("Using RDD of dict to inferSchema is deprecated. "


In [17]:
airline_rows_df.registerTempTable("airlines")

### Top 10 Best Airline (Lowest Delay)

In [18]:
results = sqlContext.sql("select avg(ArrDelay) AS delay,UniqueCarrier from airlines \
group by UniqueCarrier order by delay ASC limit 10")

In [19]:
results.show()

+------------------+-------------+
|             delay|UniqueCarrier|
+------------------+-------------+
|-2.888673890608875|           AQ|
|1.2644089394236424|           HA|
|2.8481100056939694|           US|
| 4.111134703250211|           9E|
| 4.804346362093581|           AS|
|  5.17967817300539|           WN|
| 6.108246666107523|           F9|
| 6.598884736868734|           OO|
| 7.368539129929383|           NW|
| 7.855163154883384|           DL|
+------------------+-------------+



### Top 10 Worst Airline (Highest Delay)

In [20]:
results = sqlContext.sql("select avg(ArrDelay) AS delay,UniqueCarrier from airlines \
group by UniqueCarrier order by delay DESC limit 10")

In [21]:
results.show()

+------------------+-------------+
|             delay|UniqueCarrier|
+------------------+-------------+
|12.607194035713981|           AA|
|11.817467683998007|           OH|
|11.775181433600808|           YV|
|11.291322186680183|           UA|
| 11.08418439051813|           B6|
|  10.9790372802913|           CO|
|10.635404800035221|           XE|
|10.208002415713782|           EV|
| 9.890667945776498|           MQ|
|  9.09137538507922|           FL|
+------------------+-------------+



# Data Persistence with Hive Table

In [22]:
! hdfs dfs -rm -r /user/root/fight_result

airline_rows_df.write.mode('overwrite').parquet('/user/root/fight_result')

from pyspark import HiveContext

hc = HiveContext(sc)

hc.sql("""
DROP TABLE IF EXISTS flight_result
""")

hc.sql("""
create external table IF NOT EXISTS flight_result (
ArrDelay string, UniqueCarrier string
)
STORED AS PARQUET
location "/user/root/fight_result"
""")

from time import time

t2 = time()

hc.sql('select avg(ArrDelay) AS delay,UniqueCarrier from flight_result \
group by UniqueCarrier order by delay DESC limit 10').show()

tt2 = time() - t2

tt2


Deleted /user/root/fight_result
+------------------+-------------+
|             delay|UniqueCarrier|
+------------------+-------------+
|12.607194035713981|           AA|
|11.817467683998007|           OH|
|11.775181433600808|           YV|
|11.291322186680183|           UA|
| 11.08418439051813|           B6|
|  10.9790372802913|           CO|
|10.635404800035221|           XE|
|10.208002415713782|           EV|
| 9.890667945776498|           MQ|
|  9.09137538507922|           FL|
+------------------+-------------+



4.3017518520355225